In [38]:
# 包引入
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, Batch
from sentence_transformers import SentenceTransformer

In [8]:
def generate_code_embeddings(code_data, model_name):
    """
    加载代码，生成嵌入
    """
    
    # 加载预训练模型
    print(f"Loading embedding model: {model_name}...")
    model = SentenceTransformer(model_name)

    # 生成嵌入
    print("Generating embeddings...")
    embeddings = model.encode(code_data, show_progress_bar=True).tolist()
    
    return embeddings

No sentence-transformers model found with name /home/pm/codebert-base. Creating a new one with mean pooling.


Loading embedding model: /home/pm/codebert-base...
Generating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[-0.42088693380355835,
 0.10436536371707916,
 0.34373658895492554,
 0.17476998269557953,
 -0.37004709243774414,
 -0.37455976009368896,
 -0.10668481886386871,
 0.2982756793498993,
 0.21411190927028656,
 0.4020977020263672,
 -0.23685309290885925,
 0.9128382205963135,
 -0.24038444459438324,
 -0.41095271706581116,
 0.6471357941627502,
 0.193195179104805,
 0.24080625176429749,
 0.24315451085567474,
 0.05634622275829315,
 -0.03447640687227249,
 -0.1707223802804947,
 -0.2952006459236145,
 0.5728013515472412,
 -0.6531931161880493,
 0.3403095006942749,
 0.39701181650161743,
 0.1650542914867401,
 0.8826589584350586,
 -0.32314327359199524,
 1.038062334060669,
 -0.11942604929208755,
 0.07969006896018982,
 1.663250207901001,
 -0.003997768275439739,
 0.29241180419921875,
 -0.2689550518989563,
 -0.46726828813552856,
 0.2886401414871216,
 -0.00737615255638957,
 -0.46086862683296204,
 -0.2871817946434021,
 0.9313816428184509,
 -1.048900842666626,
 0.04647719860076904,
 0.5819157361984253,
 0.3701904416

In [40]:
# 模型整体架构(必须导入，不必修改)
class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=4, dropout=0.1):
        super(CrossAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, query, key, value):
        # query, key, value: [batch_size, seq_len, embed_dim]
        attn_output, _ = self.attention(query, key, value)
        return self.norm(attn_output + query)  # Add & Norm
        
class AblationModelWithWeights(nn.Module):
    def __init__(self, 
                 code_input_dim, 
                 tabular_input_dim, 
                 gcn_hidden_dim, 
                 gcn_output_dim, 
                 embed_dim=32,
                 use_code=True, 
                 use_tabular=True, 
                 use_graph=True):
        super(AblationModelWithWeights, self).__init__()

        # 是否使用各分支
        self.use_code = use_code
        self.use_tabular = use_tabular
        self.use_graph = use_graph

        # Code Embedding 分支
        if self.use_code:
            self.code_branch = nn.Sequential(
                nn.Linear(code_input_dim, 128),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, embed_dim)
            )

        # Tabular 数据分支
        if self.use_tabular:
            self.tabular_branch = nn.Sequential(
                nn.Linear(tabular_input_dim, 64),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(64, embed_dim),
                nn.ReLU()
            )

        # GCN 分支
        if self.use_graph:
            self.gcn1 = GCNConv(32, gcn_hidden_dim)
            self.bn1 = nn.BatchNorm1d(gcn_hidden_dim)
            self.gcn2 = GCNConv(gcn_hidden_dim, gcn_output_dim)
            self.bn2 = nn.BatchNorm1d(gcn_output_dim)

        # 交叉注意力模块
        self.cross_attention_1 = CrossAttention(embed_dim)
        self.cross_attention_2 = CrossAttention(embed_dim)

        # 模态权重参数（可学习）
        self.modal_weights = nn.Parameter(torch.ones(3))  # 初始化为 1，表示每个模态的初始权重相等

        # 融合层
        self.fusion_layer = nn.Sequential(
            nn.Linear(
                (embed_dim if self.use_code else 0) +
                (embed_dim if self.use_tabular else 0) +
                (gcn_output_dim if self.use_graph else 0), 
                128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 4)  # 输出4个标签
        )

    def forward(self, code_data, tabular_data, graph_data):
        fusion_input = []
        modal_contributions = []

        # Code Embedding 分支
        if self.use_code:
            code_out = self.code_branch(code_data)  # [batch_size, embed_dim]
            fusion_input.append(code_out)
            modal_contributions.append(self.modal_weights[0])  # 添加 code 模态的权重

        # Tabular 数据分支
        if self.use_tabular:
            tabular_out = self.tabular_branch(tabular_data)  # [batch_size, embed_dim]
            fusion_input.append(tabular_out)
            modal_contributions.append(self.modal_weights[1])  # 添加 tabular 模态的权重

        # GCN 分支
        if self.use_graph:
            x, edge_index, batch = graph_data.x, graph_data.edge_index, graph_data.batch
            x = F.relu(self.bn1(self.gcn1(x, edge_index)))
            x = F.relu(self.bn2(self.gcn2(x, edge_index)))
            x = global_mean_pool(x, batch)  # [batch_size, gcn_output_dim]
            fusion_input.append(x)
            modal_contributions.append(self.modal_weights[2])  # 添加 graph 模态的权重

        # 如果没有启用任何分支，直接抛出错误
        if len(fusion_input) == 0:
            raise ValueError("At least one of 'use_code', 'use_tabular', or 'use_graph' must be True.")

        # 对模态权重进行 softmax 标准化
        modal_contributions = F.softmax(torch.stack(modal_contributions), dim=0)  # [num_modalities]
        
        # 对每个模态的输出乘以对应的权重
        weighted_fusion_input = [
            modal_contributions[i] * fusion_input[i] for i in range(len(fusion_input))
        ]

        # 融合
        fusion_input = torch.cat(weighted_fusion_input, dim=1)  # 按最后一维拼接
        out = self.fusion_layer(fusion_input)  # [batch_size, 4]

        return out

In [43]:
def test_model_by_category_with_embeddings(config, model_path, code_embeddings, device):
    # 初始化模型
    model = AblationModelWithWeights(
        code_input_dim=768,  # 输入为预先生成的嵌入维度
        tabular_input_dim=16,
        gcn_hidden_dim=32,
        gcn_output_dim=16,
        embed_dim=32,
        **config
    ).to(device)

    # 加载模型参数
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 模态处理逻辑
    with torch.no_grad():
        # 直接处理整个 code_embeddings
        code_data = torch.tensor(code_embeddings).unsqueeze(0).to(device)  # 生成 [1, 768] 的张量
        tabular_data = torch.zeros(code_data.size(0), 16).to(device)  # 填充 tabular 数据为 (batch_size, 16)
        graph_data = Data(
            x=torch.zeros(code_data.size(0), 32).to(device),  # 填充节点特征为 32
            edge_index=torch.zeros(2, code_data.size(0)).to(device).long(),  # 填充边索引
            batch=torch.zeros(code_data.size(0), dtype=torch.long).to(device)  # 填充批次索引
        )
        # 模型推理
        outputs = model(code_data, tabular_data, graph_data)
    return outputs.cpu()

In [46]:
# 训练设备部署
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

'''
codebert-base需要安装

1、安装huggingface-cli（使用镜像）
pip install 'huggingface_hub[cli]' --index-url=https://mirrors.aliyun.com/pypi/simple

2、下载codebert-base模型
huggingface-cli download microsoft/codebert-base --local-dir 预存储本地位置

'''

# 接收输入的代码
source_code= "__global__ void mm2_kernel1(int ni, int nj, int nk, int nl, DATA_TYPE alpha, DATA_TYPE beta, DATA_TYPE *tmp, DATA_TYPE *A, DATA_TYPE *B)\n{\n\tint j = blockIdx.x * blockDim.x + threadIdx.x;\n\tint i = blockIdx.y * blockDim.y + threadIdx.y;\n\n\tif ((i < _PB_NI) && (j < _PB_NJ))\n\t{ \n\t\ttmp[i * NJ + j] = 0;\n\t\tint k;\n\t\tfor (k = 0; k < _PB_NK; k++)\n\t\t{\n\t\t\ttmp[i * NJ + j] += alpha * A[i * NK + k] * B[k * NJ + j];\n\t\t}\n\t}\n}"

# 替换为codebert-base本地存储位置
embedding_model = "/home/pm/codebert-base"

# 输入代码，生成嵌入
code_embedding = generate_code_embeddings(source_code, embedding_model)

# 存储所有预测结果
all_predictions = []

# 设置模态
configs = [
    {'use_code': True, 'use_tabular': True, 'use_graph': True},  # 全部启用（Baseline）
    {'use_code': False, 'use_tabular': True, 'use_graph': True},  # 去掉 Code 分支
    {'use_code': True, 'use_tabular': False, 'use_graph': True},  # 去掉 Tabular 分支
    {'use_code': True, 'use_tabular': True, 'use_graph': False},  # 去掉 Graph 分支
    {'use_code': True, 'use_tabular': False, 'use_graph': False},  # 只有code 分支
]
for config in configs:
    # 模型加载
    model_load_path = f"model/model_{str(config)}.pth"
    # 预测
    prediction=test_model_by_category_with_embeddings(config, model_load_path, code_embedding, device)
    all_predictions.append(prediction)
    
# 参数名称
parameter_names = ["AMAT", "Time", "L1 Hit Rate", "L2 Hit Rate"]

# 提取并打印每个预测的结果
for idx, prediction in enumerate(all_predictions):
    print(f"Prediction {idx + 1}:")
    for param_idx, param_name in enumerate(parameter_names):
        print(f"  {param_name}: {prediction[0][param_idx]:.4f}")
    print()  # 空行分隔每个预测结果

Prediction 1:
  AMAT: 0.0028
  Time: 0.0330
  L1 Hit Rate: 0.6216
  L2 Hit Rate: -0.0050

Prediction 2:
  AMAT: 0.0004
  Time: 0.0391
  L1 Hit Rate: 0.6577
  L2 Hit Rate: 0.0046

Prediction 3:
  AMAT: 0.0660
  Time: -0.0055
  L1 Hit Rate: 0.9032
  L2 Hit Rate: 1.2431

Prediction 4:
  AMAT: 0.0081
  Time: 0.0345
  L1 Hit Rate: 0.6241
  L2 Hit Rate: 0.0013

Prediction 5:
  AMAT: 0.0002
  Time: 0.0402
  L1 Hit Rate: 0.6039
  L2 Hit Rate: 0.0391

